# 🧩 `parse_markdown_examples_to_json(text: str)`

Парсит текст с примерами, содержащими блоки `**Input:**` и `**Output:**`, и возвращает их в виде словаря. Этот формат используется у LeetCode.

---

## 📥 Аргументы

| Параметр | Тип   | Описание                             |
|----------|-------|--------------------------------------|
| `text`   | `str` | Текст, содержащий примеры в формате: |
| -        |       | `**Example N:**`                     |
| -        |       | `**Input:** <входные данные>`        |
| -        |       | `**Output:** <выходные данные>`      |

---

## 📤 Возвращает

Тип: `dict`

Структура возвращаемого словаря:

```json
{
  "examples": [
    {
      "input": {...},     // Входные данные (dict)
      "output": ...       // Выходные данные (любой тип)
    }
  ]
}
```

---


# 🔻 Вставьте пример для конвертирования в JSON

In [54]:
example_block = """
**Example 1:**

![](https://assets.leetcode.com/uploads/2018/12/14/binarysearchtree_improved.png)

**Input:** root = [6,2,8,0,4,7,9,null,null,3,5], p = 2, q = 8
**Output:** 6
**Explanation:** The LCA of nodes 2 and 8 is 6.

**Example 2:**

![](https://assets.leetcode.com/uploads/2018/12/14/binarysearchtree_improved.png)

**Input:** root = [6,2,8,0,4,7,9,null,null,3,5], p = 2, q = 4
**Output:** 2
**Explanation:** The LCA of nodes 2 and 4 is 2, since a node can be a descendant of itself according to the LCA definition.

**Example 3:**

**Input:** root = [2,1], p = 2, q = 1
**Output:** 2
"""

## 1. Функция-обработчик

In [55]:
import re
import ast
import json

def parse_markdown_examples_to_json(text: str):
    examples = []
    raw_blocks = re.split(r"\*\*Example \d+:\*\*", text)

    for idx, block in enumerate(raw_blocks, 1):
        prefix = f"Example {idx}"

        # Попытка найти input
        input_lines = re.findall(r"\*\*Input:\*\*\s*(.*?)(?=\n\*\*|$)", block, re.DOTALL)
        if not input_lines:
            continue

        input_lines = [line.strip().rstrip(",") for line in input_lines[0].strip().splitlines() if line.strip()]
        input_text = "\n".join(input_lines)

        # Заменяем JSON-значения на Python-эквиваленты
        input_text = re.sub(r"\bnull\b", "None", input_text)
        input_text = re.sub(r"\btrue\b", "True", input_text)
        input_text = re.sub(r"\bfalse\b", "False", input_text)

        try:
            if len(input_lines) == 1 and '=' in input_lines[0]:
                # Пример: root = [...]
                input_expr = "{" + re.sub(r"(\w+)\s*=", r'"\1":', input_text) + "}"
                input_data = ast.literal_eval(input_expr)
            else:
                # Пример: ["MyQueue"], [[], []]
                input_data = {"args": [ast.literal_eval(line) for line in input_lines]}
        except Exception as e:
            print(f"{prefix} ⚠ Ошибка в input: {e}")
            continue

        # Попытка найти output
        output_match = re.search(r"\*\*Output:\*\*\s*(.*?)\s*(?:\n|$)", block)
        if not output_match:
            continue

        output_text = output_match.group(1).strip()
        output_text = output_text.replace("null", "None").replace("true", "True").replace("false", "False")

        try:
            output_data = ast.literal_eval(output_text)
        except Exception as e:
            print(f"{prefix} ⚠ Ошибка в output: {e}")
            continue

        examples.append({
            "input": input_data,
            "output": output_data
        })

    return {"examples": examples}


## 2. Парсим примеры из текста

In [56]:
parsed = parse_markdown_examples_to_json(example_block)
print(parsed)

{'examples': [{'input': {'root': [6, 2, 8, 0, 4, 7, 9, None, None, 3, 5], 'p': 2, 'q': 8}, 'output': 6}, {'input': {'root': [6, 2, 8, 0, 4, 7, 9, None, None, 3, 5], 'p': 2, 'q': 4}, 'output': 2}, {'input': {'root': [2, 1], 'p': 2, 'q': 1}, 'output': 2}]}


## 3. Преобразуем результат в JSON-строку с красивым форматированием

In [57]:
pretty = json.dumps(parsed, indent=2, ensure_ascii=False)
print(pretty)

{
  "examples": [
    {
      "input": {
        "root": [
          6,
          2,
          8,
          0,
          4,
          7,
          9,
          null,
          null,
          3,
          5
        ],
        "p": 2,
        "q": 8
      },
      "output": 6
    },
    {
      "input": {
        "root": [
          6,
          2,
          8,
          0,
          4,
          7,
          9,
          null,
          null,
          3,
          5
        ],
        "p": 2,
        "q": 4
      },
      "output": 2
    },
    {
      "input": {
        "root": [
          2,
          1
        ],
        "p": 2,
        "q": 1
      },
      "output": 2
    }
  ]
}


## 4. Убираем лишние пробелы и переносы строк внутри массивов

In [58]:
pretty = re.sub(r'\[\s+([^\[\]]+?)\s+\]', lambda m: '[' + m.group(1).replace('\n', '').replace(' ', '') + ']', pretty)
print(pretty)

{
  "examples": [
    {
      "input": {
        "root": [6,2,8,0,4,7,9,null,null,3,5],
        "p": 2,
        "q": 8
      },
      "output": 6
    },
    {
      "input": {
        "root": [6,2,8,0,4,7,9,null,null,3,5],
        "p": 2,
        "q": 4
      },
      "output": 2
    },
    {
      "input": {
        "root": [2,1],
        "p": 2,
        "q": 1
      },
      "output": 2
    }
  ]
}


# 🔺 Заберите результат в буфер обмена